# Load datasets
---

In [1]:
import datasets
from xsum_dataset import XsumDataset

In [2]:
xsum_data_raw = datasets.load_dataset("xsum")

Using custom data configuration default
Reusing dataset xsum (/home/wk247/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# train/val/test data
# xsum_train_data = XsumDataset(xsum_data_raw["train"])
xsum_val_data = XsumDataset(xsum_data_raw["validation"])
xsum_test_data = XsumDataset(xsum_data_raw["test"])

# concat data
xsum_data_raw_cc = datasets.concatenate_datasets(
    [xsum_data_raw["train"], xsum_data_raw["validation"], xsum_data_raw["test"]]
    )
xsum_concat_data = XsumDataset(xsum_data_raw_cc)

## select a sample

In [4]:
from generate_xsum_summary import load_summarization_model_and_tokenizer, generate_summaries, generate_token_entropy_metadata
import random

random.seed(0)

### * one to be perturbed

In [5]:
# sample one bbcid
bbc_id = random.choice(list(xsum_test_data.data_by_id.keys()))
# or fix one: bbc_ids = ["33858956"]
bbc_id

'35616768'

In [6]:
# selected_data - dict with keys: (id, document, true_summary, (factuality_data, faithfulness_data))
selected_data = xsum_test_data.data_by_id[bbc_id]

# original_docs - documents to sumamrize
original_doc = selected_data["document"]
print("original doc to summarize:\n", original_doc)

original doc to summarize:
 The agreement, reached late on Friday after two days of talks in Brussels, gives the UK power to limit some EU migrants' benefits.
It also includes a treaty change so the UK is not bound to "ever closer union" with other EU member states, he said.
EU exit campaigners said the "hollow" deal offered only "very minor changes".
Mr Cameron is set to the announce the date of a referendum on whether Britain should remain in the EU after a cabinet meeting which is happening at 10:00 GMT - the referendum is widely expected to be on Thursday, 23 June.
Once the date is announced, ministers will be allowed to campaign for whichever side they want - one of Mr Cameron's closest political allies Michael Gove has already been named as supporting the Leave camp. Others, such as Iain Duncan Smith are expected to follow - but a question mark remains over which way London Mayor Boris Johnson will jump.
The key points of the deal are:
The prime minster had to make concessions to

### *one to perturb

In [4]:
import random

In [5]:
# sample one bbcid
ptb_id = random.choice(xsum_val_data.ids)
print("ptb_id:", ptb_id)

ptb_id: 29703870


In [42]:
# selected_data - dict with keys: (id, document, true_summary, (factuality_data, faithfulness_data))
ptb_selected_data = xsum_val_data.data_by_id[ptb_id]

# ptb_docs - documents to sumamrize
ptb_doc = ptb_selected_data["document"]
print("ptb_doc to sample ood sentences:\n", ptb_doc)

ptb_doc to sample ood sentences:
 Tait, who picked up a medal alongside Kate Hornsey in the women's pair, was diagnosed with cervical cancer in 2013.
Australian Rowing head coach Chris O'Brien said: "Sarah has been, and will continue to be, an inspiration to many Australian rowers.
"She was a strong role model to young athletes, and particularly influential with our emerging female athletes."
Tait and Hornsey finished behind Great Britain's Heather Stanning and Helen Glover in the 2012 pairs final, which was GB's first gold of London 2012.
Glover became friends with Tait following the final and told BBC World Service she was an "incredible personality".
She added: "To achieve so much in her too short life - two beautiful children, a husband, an Olympic silver medal, the list goes on.
"Above all that, she was an inspirational and truly lovely person that has paved the way for women in sport at home in Australia and all over the world."


### * insert sents

In [43]:
sample_sents_n = 2

In [44]:
ptb_sentences = random.choices(ptb_doc.split("\n"), k=sample_sents_n)
print(ptb_sentences)

['Australian Rowing head coach Chris O\'Brien said: "Sarah has been, and will continue to be, an inspiration to many Australian rowers.', 'She added: "To achieve so much in her too short life - two beautiful children, a husband, an Olympic silver medal, the list goes on.']


In [45]:
original_sents = original_doc.split("\n")
len(original_sents)

35

In [46]:
for ptb_sent in ptb_sentences:
    insert_idx = random.choice(range(len(original_sents)))
    print("insert_idx", insert_idx)
    original_sents.insert(insert_idx, ptb_sent)

insert_idx 32
insert_idx 20


In [47]:
len(original_sents)

37

In [48]:
ood_doc = u"\n".join(original_sents)

## generate summary from ood document

### load model and tokenizer

In [61]:
model_name = "facebook/bart-large-xsum"
model, tokenizer = load_summarization_model_and_tokenizer(model_name)

In [62]:
import time

In [71]:
beam_size = 100

In [72]:
# generate summaries and metadata
start = time.time()
gen_summaries, gen_metadata = generate_summaries(
    model,
    tokenizer,
    original_doc,
    num_beams=beam_size,
    return_generation_metadata=True
)
end = time.time()
print(f"time: {end - start}")

time: 3.0090742111206055


In [73]:
# generate summaries and metadata
start = time.time()
ood_summaries, ood_metadata = generate_summaries(
    model,
    tokenizer,
    ood_doc,
    num_beams=beam_size,
    return_generation_metadata=True
)
end = time.time()
print(f"time: {end - start}")

time: 2.862010955810547


In [74]:
gen_metadata

[[{'token_id': tensor(37703, device='cuda:0'),
   'token': 'Twenty',
   'entropy': 5.396252632141113,
   'beam_token_prob': 0.06844499707221985,
   'beam_idx': 0,
   'beam_top_probs': [{'token': 'He',
     'token_id': 894,
     'beam_token_prob': 0.0970643162727356},
    {'token': 'The', 'token_id': 133, 'beam_token_prob': 0.08087211847305298},
    {'token': 'It', 'token_id': 243, 'beam_token_prob': 0.0751330628991127}],
   'token_in_input': True},
  {'token_id': tensor(107, device='cuda:0'),
   'token': ' years',
   'entropy': 1.4511500597000122,
   'beam_token_prob': 0.8674792051315308,
   'beam_idx': 4,
   'beam_top_probs': [{'token': ' years',
     'token_id': 107,
     'beam_token_prob': 0.8674792051315308},
    {'token': '-', 'token_id': 12, 'beam_token_prob': 0.027631521224975586},
    {'token': ' two',
     'token_id': 80,
     'beam_token_prob': 0.0023367642425000668}],
   'token_in_input': True},
  {'token_id': tensor(536, device='cuda:0'),
   'token': ' ago',
   'entropy': 2

In [ ]:
# generate token entropy metadata
gen_token_entropy_metadata = generate_token_entropy_metadata(bbc_ids, gen_metadata)

In [ ]:
# selected_data - list of dicts with keys: (id, document, true_summary, (factuality_data, faithfulness_data))
selected_data = [xsum_test_data.data_by_id[x] for x in bbc_ids]

# original_docs - list of documents to sumamrize
original_docs = [x["document"] for x in selected_data]
print("original docs to summarize:\n", original_docs)

In [9]:
original_docs[0].split("\n")

["Sian O'Callaghan, 22, was last seen leaving Suju alone at about 0250 GMT on Saturday to walk half a mile home.",
 'About 400 people took part in the search of Savernake Forest near Marlborough which is due to resume at 1030 GMT on Wednesday.',
 'A £20,000 reward to help find her has been offered by an anonymous donor.',
 'Mobile records put her phone in the area of the 4,500-acre forest 32 minutes after she left the club.',
 'Police say the journey could only have been made by car.',
 "Miss O'Callaghan's friends and her boyfriend Kevin Reape joined the search along with local people, many of whom did not know the missing woman, following a police appeal for help.",
 'Volunteers searching the dense woodland were divided into groups which were then assigned to a specialist police officer.',
 'Mikey Jack, 19, who plays Sunday league football with Mr Reape, said: "A lot of people here don\'t really know Kevin or Sian but they just want to help any way they can."',
 'Another volunteer, co

# NER Analysis
---

In [5]:
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_lg")

## 0-0) file save / load utils

In [6]:
import pickle
import os

In [7]:
def save_to_cache_dir(var, file_name):
    cache_path = "../cache"
    file_path = os.path.join(cache_path, file_name + ".pkl")
    with open(file_path, "wb") as f:
        pickle.dump(var, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"saved to '{file_path}'")

In [8]:
def load_from_cache_dir(file_name):
    cache_path = "../cache"
    file_path = os.path.join(cache_path, file_name + ".pkl")
    with open(file_path, 'rb') as f:
        var = pickle.load(f)
    print(f"'{file_path}' loaded")
    return var

## 0-1) NER utils

In [9]:
# function to display basic entity info: 
def show_ents(doc):
    print(f"original doc: {doc.text}")
    if doc.ents: 
        for ent in doc.ents: 
            print(f"entity: {ent.text : >13} | start_char: {ent.start_char: 3} | end_char: {ent.end_char: 3} | label: {ent.label_} - {spacy.explain(ent.label_)}")
            # print(ent.label)  # entity type's hash value
            # print(ent.start)  # token span's start index position (word index)
            # print(ent.end)  # token span's stop index position (word index)
    else: print('No named entities found.')

## 0-2) load cached NER files

In [10]:
# # train
# train_doc_ents_list = load_from_cache_dir("train_doc_ents_list")
# train_sum_ents_list = load_from_cache_dir("train_sum_ents_list")

# # val
# val_doc_ner_list = load_from_cache_dir("val_doc_ner_list")
# val_sum_ner_list = load_from_cache_dir("val_sum_ner_list")
# val_doc_ents_list = load_from_cache_dir("val_doc_ents_list_pp")  # preprocessed
# val_sum_ents_list = load_from_cache_dir("val_sum_ents_list_pp")

# test
test_doc_ner_list = load_from_cache_dir("test_doc_ner_list")
test_sum_ner_list = load_from_cache_dir("test_sum_ner_list")
test_doc_ents_list = load_from_cache_dir("test_doc_ents_list_pp")  # preprocessed
test_sum_ents_list = load_from_cache_dir("test_sum_ents_list_pp")

# entities pool
concat_ent_pool_dict = load_from_cache_dir("concat_ent_pool_dict_pp")  # preprocessed
# test_ent_pool_dict = load_from_cache_dir("test_ent_pool_dict")

'../cache/test_doc_ner_list.pkl' loaded
'../cache/test_sum_ner_list.pkl' loaded
'../cache/test_doc_ents_list_pp.pkl' loaded
'../cache/test_sum_ents_list_pp.pkl' loaded
'../cache/concat_ent_pool_dict_pp.pkl' loaded


## todo
* case 1. no entites in true_summary - continue
    1) ner on summary doesn't recognize the entity correctly
* case 2. no exact overlap entities btw true_summary - document   
    1) check subset
        * Edinburgh (summary) - Edinburgh Sheriff Court (document)
        * (name) Usain Bolt (summary) - Usain (document) - should I replace?
    
* case 3. ex) 4 - four
* case 4. case

1. how to choose the entity?
    * pick the replacement entity that is not in the source document
2. how to replace it?





### other ideas
* replace the entites that are not in the true summary?

## 1) analyze test summaries

In [11]:
data_idx = 0

In [12]:
sum_ner = test_sum_ner_list[data_idx]
doc_ner = test_doc_ner_list[data_idx]

In [13]:
doc_ents_sorted = test_doc_ents_list[data_idx].most_common()
sum_ents_sorted = test_sum_ents_list[data_idx].most_common()

In [14]:
show_ents(sum_ner)

original doc: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
entity:         Wales | start_char:  65 | end_char:  70 | label: GPE - Countries, cities, states


In [15]:
sum_ents_sorted

[(('Wales', 'GPE'), 1)]

In [16]:
doc_ents_sorted

[(('one', 'CARDINAL'), 4),
 (('Wales', 'GPE'), 2),
 (('Marc', 'PERSON'), 2),
 (('1,099', 'CARDINAL'), 1),
 (('2015-16', 'DATE'), 1),
 (('up to a year', 'DATE'), 1),
 (('The Welsh Government', 'ORG'), 1),
 (('the Housing Act', 'LAW'), 1),
 (('2015', 'DATE'), 1),
 (('Andrew Stevens', 'PERSON'), 1),
 (('six months to a year', 'DATE'), 1),
 (('six months', 'DATE'), 1),
 (('Stevens', 'PERSON'), 1),
 (('a hundred pounds', 'MONEY'), 1),
 (('830', 'CARDINAL'), 1),
 (('the year to March 2016', 'DATE'), 1),
 (('6,900', 'CARDINAL'), 1),
 (('50', 'DATE'), 1),
 (('the past 20 years', 'DATE'), 1),
 (('Tom Clarke', 'PERSON'), 1),
 (('Emmaus South Wales', 'GPE'), 1),
 (('Emmaus', 'ORG'), 1),
 (('Welsh Government', 'ORG'), 1),
 (('20,000', 'CARDINAL'), 1),
 (('the next five years', 'DATE'), 1)]

## 2) filter entities

* label list
    * CARDINAL: Numerals that do not fall under another type
    * DATE: Absolute or relative dates or periods
    * EVENT: Named hurricanes, battles, wars, sports events, etc.
    * FAC: Buildings, airports, highways, bridges, etc.
    * GPE: Countries, cities, states
    * LANGUAGE: Any named language
    * LAW: Named documents made into laws.
    * LOC: Non-GPE locations, mountain ranges, bodies of water
    * MONEY: Monetary values, including unit
    * NORP: Nationalities or religious or political groups
    * ORDINAL: "first", "second", etc.
    * ORG: Companies, agencies, institutions, etc.
    * PERCENT: Percentage, including "%"
    * PERSON: People, including fictional
    * PRODUCT: Objects, vehicles, foods, etc. (not services)
    * QUANTITY: Measurements, as of weight or distance
    * TIME: Times smaller than a day
    * WORK_OF_ART: Titles of books, songs, etc.

FILTER_LABELS = ["PERSON", "FAC", "GPE", "NORP", "LOC", "EVENT"]

In [11]:
FILTER_LABELS = ["PERSON", "FAC", "GPE", "NORP", "LOC", "EVENT", "LANGUAGE", "LAW", "ORG"]
# without numbers? "DATE", "PERCENT" 
# too many errors: "PRODUCT", "WORK_OF_ART"

In [12]:
ALL_LABELS = list(NER.get_pipe('ner').labels)

## 3) replace entities

In [13]:
from tqdm.notebook import tqdm

### 1) count samples to exclude

In [14]:
def count_no_overlap(xsum_data, doc_ents_list, sum_ents_list, filter_labels):
    # samples to exclude
    doc_overlap_count = 0
    sum_overlap_count = 0
    no_overlap_count = 0

    for data_idx, data in enumerate(tqdm(xsum_data.dataset)):
        document = data["document"]
        true_summary = data["true_summary"]

        # ner
        # doc_ner = doc_ner_list[data_idx]
        # sum_ner = sum_ner_list[data_idx]

        # entites
        doc_ents = doc_ents_list[data_idx]
        sum_ents = sum_ents_list[data_idx]

        # sort and filter
        doc_ents_filtered_sorted = [((ent, label), count) for ((ent, label), count) in doc_ents.most_common() 
                                    if label in filter_labels]
        sum_ents_filtered_sorted = [((ent, label), count) for ((ent, label), count) in sum_ents.most_common() 
                                    if label in filter_labels]

        overlap_flag = False
        # from document side
        for (ent, label), count in doc_ents_filtered_sorted:
            if ent in true_summary:  # overlap exists
                overlap_flag = True
                doc_overlap_count += 1
                break

        # if no overlap from document side, try from summary
        if overlap_flag == False:
            if len(sum_ents) == 0: # no entity in summary -> pass
                pass
            else:
                for (ent, label), count in sum_ents_filtered_sorted:
                    if ent in document:
                        overlap_flag = True
                        sum_overlap_count += 1
                        break

        if overlap_flag == False:
            no_overlap_count += 1 # need to exclude

    # check uncounted sample
    assert doc_overlap_count + sum_overlap_count + no_overlap_count == len(xsum_data.dataset)
    
    return doc_overlap_count, sum_overlap_count, no_overlap_count

In [15]:
test_overlap_counts = count_no_overlap(xsum_data=xsum_test_data, 
                                       doc_ents_list=test_doc_ents_list, 
                                       sum_ents_list=test_sum_ents_list, 
                                       filter_labels=FILTER_LABELS)

  0%|          | 0/11334 [00:00<?, ?it/s]

In [16]:
n_test = len(xsum_test_data.dataset)
print(f"overlap from document: {(test_overlap_counts[0]/n_test)*100:.1f}%")
print(f"overlap from summary: {(test_overlap_counts[1]/n_test)*100:.1f}%")
print(f"no overlap: {(test_overlap_counts[2]/n_test)*100:.1f}%")

overlap from document: 72.6%
overlap from summary: 3.6%
no overlap: 23.7%


### 2) print

In [17]:
# extract only necessary dicts
for label in ALL_LABELS:
    if label not in FILTER_LABELS:
        del concat_ent_pool_dict[label]

In [18]:
assert(len(concat_ent_pool_dict) == len(FILTER_LABELS))

In [19]:
# reduce the size to 5%
reduced_concat_ent_pool_dict = {}
for label in concat_ent_pool_dict.keys():
    ent_pool = concat_ent_pool_dict[label]
    reduced_ent_pool = {k:v for (k,v) in list(ent_pool.items())[:len(ent_pool) // 10]}
    reduced_concat_ent_pool_dict[label] = reduced_ent_pool

In [20]:
len(concat_ent_pool_dict["PERSON"])

358369

In [21]:
len(reduced_concat_ent_pool_dict["PERSON"])

35836

In [22]:
import random
random.seed(0)

In [25]:
chosen_ent_label_list = []
for data_idx, data in enumerate(tqdm(xsum_test_data.dataset)):
    print(f"\n============ data idx: {data_idx} ============")
    document = data["document"]
    true_summary = data["true_summary"]
    
    # ner
    # doc_ner = test_doc_ner_list[data_idx]
    # sum_ner = test_sum_ner_list[data_idx]
    
    # entites
    doc_ents = test_doc_ents_list[data_idx]
    sum_ents = test_sum_ents_list[data_idx]
    
    # sort and filter
    doc_ents_filtered_sorted = [((ent, label), count) for ((ent, label), count) in doc_ents.most_common() 
                                if label in FILTER_LABELS]
    sum_ents_filtered_sorted = [((ent, label), count) for ((ent, label), count) in sum_ents.most_common() 
                                if label in FILTER_LABELS]
    
    # if there is an overlap
    overlap_flag = False
    
    
    chosen_ent_label = None
    
    # from document side
#     print(f"* summary: {true_summary}")
#     print(f"* document ents: {doc_ents_filtered_sorted} \n")
    for (ent, label), count in doc_ents_filtered_sorted:
        if ent in true_summary:  # overlap exists
#             print("** overlap from document")
#             print(f"ent: {ent}, label: {label}, count_doc: {count}, count_sum: {true_summary.count(ent)}")
            overlap_flag = True
            chosen_ent_label = (ent, label)
            break
    
    # if no overlap from document side, try from summary
    if overlap_flag == False:
        if len(sum_ents) == 0: # no entity in summary -> pass
            pass
        else:
#             print(f"* summary ents: {sum_ents_filtered_sorted} \n")
            for (ent, label), count in sum_ents_filtered_sorted:
                if ent in document:
#                     print("** overlap from summary")
#                     print(f"ent: {ent}, label: {label}, count_sum: {count}, count_doc: {document.count(ent)}")
                    overlap_flag = True
                    chosen_ent_label = (ent, label)
                    break
    
    
    # check the chosen entity
    if overlap_flag == True:
        chosen_ent_label_list.append(chosen_ent_label)
        
        chosen_ent, chosen_label = chosen_ent_label
        print(f"* summary: {true_summary}")
        print(f"* chosen_ent: {chosen_ent}, label: {chosen_label}")
        
        
        # choose one
        ent_pool = list(reduced_concat_ent_pool_dict[chosen_label].keys())
        replacement = random.choice(ent_pool)
        print(f"* replacement: {replacement}")
        while len(replacement.split()) != len(chosen_ent.split()):  # also this should not be in doc or summary
            print("pick new replacement")
            replacement = random.choice(ent_pool)
            print(f"* replacement: {replacement}")
        
    else:
        print("****** NO OVERLAP ******")
        chosen_ent_label_list.append((None, None))

  0%|          | 0/11334 [00:00<?, ?it/s]


============ data idx: 0 ============
* summary: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
* chosen_ent: Wales, label: GPE
* replacement: Qingdao

============ data idx: 1 ============
* summary: A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh.
* chosen_ent: Edinburgh, label: GPE
* replacement: Dunblane

============ data idx: 2 ============
* summary: Four people accused of kidnapping and torturing a mentally disabled man in a "racially motivated" attack streamed on Facebook have been denied bail.
* chosen_ent: Facebook, label: ORG
* replacement: Stam

============ data idx: 3 ============
* summary: West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading.
* chosen_ent: West Brom, label: GPE
* replacement: the Irish Republic
pick new replacement
* replacement: Gymraeg
pick new replacement
* replacement: Northwich
pick new replaceme

pick new replacement
* replacement: Frank McParland
pick new replacement
* replacement: Shaq Coulthirst
pick new replacement
* replacement: McCaw

============ data idx: 291 ============
* summary: Leicester's decision to sack Claudio Ranieri nine months after winning the Premier League made former Foxes striker Gary Lineker "shed a tear".
* chosen_ent: Leicester, label: GPE
* replacement: St Ann's
pick new replacement
* replacement: Hoffenheim

============ data idx: 292 ============
* summary: Rescuers are frantically trying to save about nine people located in the wreckage of a collapsed factory complex in the Bangladeshi capital Dhaka.
* chosen_ent: Dhaka, label: GPE
* replacement: Newport West
pick new replacement
* replacement: Warrington

============ data idx: 293 ============
* summary: Dundee United have signed their second goalkeeper in a week by bringing in Harry Lewis on loan from Southampton.
* chosen_ent: Lewis, label: PERSON
* replacement: Danny Kent
pick new replacemen

* replacement: Muadhamiya

============ data idx: 601 ============
* summary: A 1969 Led Zeppelin session for the BBC that was thought to have been lost when archives were wiped has been recovered from a recording made by a fan.
* chosen_ent: BBC, label: ORG
* replacement: The Portadown Times
pick new replacement
* replacement: The Church
pick new replacement
* replacement: Aegon

============ data idx: 602 ============
* summary: As the Welsh government publishes plans to reintroduce Welsh taxes for the first time since the 13th century, BBC News looks at what life was like in Wales last time there was direct Welsh taxation.
* chosen_ent: Wales, label: GPE
* replacement: Mercer

============ data idx: 603 ============
****** NO OVERLAP ******

============ data idx: 604 ============
****** NO OVERLAP ******

============ data idx: 605 ============
* summary: Northern Ireland's new health minister has challenged politicians to accept change as he outlined his vision for the future of l

* replacement: Withington

============ data idx: 908 ============
* summary: England made the right decision to omit some of their Premier League players in the European Under-21 Championship, says Football Association director of elite development Dan Ashworth.
* chosen_ent: England, label: GPE
* replacement: Balham

============ data idx: 909 ============
* summary: At his victory rally in New York US president-elect Donald Trump promised "great, great relationships" with other nations.
* chosen_ent: Trump, label: PERSON
* replacement: Neil Gorsuch
pick new replacement
* replacement: Van der Bellen
pick new replacement
* replacement: Bernard Matthews
pick new replacement
* replacement: Bowman

============ data idx: 910 ============
****** NO OVERLAP ******

============ data idx: 911 ============
* summary: Leicester director of rugby Richard Cockerill believes other clubs are offering big money to Manu Tuilagi.
* chosen_ent: Manu, label: PERSON
* replacement: Mark Griffin
pick new

...

* chosen_ent: UK, label: GPE
* replacement: Airdrie

============ data idx: 10197 ============
* summary: A four-year-old boy has died in an incident at a farm in Maguiresbridge, County Fermanagh.
* chosen_ent: Maguiresbridge, label: PERSON
* replacement: Charlie Broadway
pick new replacement
* replacement: Witold Waszczykowski
pick new replacement
* replacement: Hammerstein

============ data idx: 10198 ============
* summary: South Africa's prosecutors have sought permission to appeal against athlete Oscar Pistorius' "shockingly light" sentence, court papers show.
* chosen_ent: Pistorius, label: PERSON
* replacement: Aled Brew
pick new replacement
* replacement: Jez George
pick new replacement
* replacement: Stephen Myler
pick new replacement
* replacement: Craig Noone
pick new replacement
* replacement: Javeed

============ data idx: 10199 ============
* summary: The owners of Dunsfold Park aerodrome, which is home to the BBC's Top Gear, have lost their latest fight for unrestricted

* replacement: PPF
pick new replacement
* replacement: Sanjoy MajumderBBC News
pick new replacement
* replacement: Sharia
pick new replacement
* replacement: the Care Quality Commission's
pick new replacement
* replacement: Haram
pick new replacement
* replacement: RedState
pick new replacement
* replacement: Concord Rangers

============ data idx: 10394 ============
* summary: Sussex's victory bid was frustrated as David Lloyd's unbeaten hundred enabled Glamorgan to bat out the final day.
* chosen_ent: Sussex, label: ORG
* replacement: Birch

============ data idx: 10395 ============
* summary: The head of one of the UK's largest credit unions is to become chief executive of Airdrie Savings Bank.
* chosen_ent: Airdrie Savings Bank, label: ORG
* replacement: Phoenix Suns
pick new replacement
* replacement: Granada Studios
pick new replacement
* replacement: Invictus
pick new replacement
* replacement: P Durcan
pick new replacement
* replacement: Corriere Della Sera

============ data i

****** NO OVERLAP ******

============ data idx: 10654 ============
* summary: Poet Owen Sheers has won Wales Book of the Year at a ceremony in Caernarfon for his work about three young soldiers.
* chosen_ent: Wales, label: GPE
* replacement: Guinea-Bissau

============ data idx: 10655 ============
* summary: England Lions batsmen Ben Duckett and Daniel Bell-Drummond could play for England within a year, according to former national coach Andy Flower.
* chosen_ent: Lions, label: ORG
* replacement: BBC Africa's
pick new replacement
* replacement: Orange

============ data idx: 10656 ============
****** NO OVERLAP ******

============ data idx: 10657 ============
****** NO OVERLAP ******

============ data idx: 10658 ============
* summary: Formula 1 boss Bernie Ecclestone says there is "no reason" why this weekend's Bahrain Grand Prix should not go ahead.
* chosen_ent: Bahrain, label: GPE
* replacement: Lancaster

============ data idx: 10659 ============
* summary: Downton Abbey star M


============ data idx: 10997 ============
****** NO OVERLAP ******

============ data idx: 10998 ============
* summary: The government has overturned a decision by Bristol City Council to refuse planning permission for a McDonald's drive-through.
* chosen_ent: McDonald, label: ORG
* replacement: Denali

============ data idx: 10999 ============
* summary: Mental health patients are being placed into overstretched A&E departments as police crackdown on the number locked up in their cells, it has been claimed.
* chosen_ent: A&E, label: ORG
* replacement: the Crumlin Ardoyne Residents Association
pick new replacement
* replacement: Barclays Bank
pick new replacement
* replacement: Euroscepticism

============ data idx: 11000 ============
* summary: Tiger Woods is back in upbeat mood and expecting to win major championships over the next decade.
* chosen_ent: Woods, label: ORG
* replacement: Vancouver Whitecaps
pick new replacement
* replacement: the US Centers for Disease Control and Pr

* replacement: The Super Falcons
pick new replacement
* replacement: the University Hospital
pick new replacement
* replacement: Royal Mail
pick new replacement
* replacement: Llambias

============ data idx: 11202 ============
* summary: There have been a number of attacks in the city of Brussels, the capital of Belgium.
* chosen_ent: Brussels, label: GPE
* replacement: Rio Ave
pick new replacement
* replacement: San Bernardino
pick new replacement
* replacement: Ferryhill

============ data idx: 11203 ============
* summary: A former Russian army officer who is alleged to have fought for the Taliban in Afghanistan has appeared in court in the United States on terrorism charges.
* chosen_ent: Afghanistan, label: GPE
* replacement: Georgia

============ data idx: 11204 ============
****** NO OVERLAP ******

============ data idx: 11205 ============
* summary: Austin Macphee could be the only Scotsman at Euro 2016 after being part of the coaching set-up at Northern Ireland.
* chosen_ent

In [ ]:
chosen_ent_list

In [63]:
# chosen_ent_groupby_label
chosen_ent_groupby_label = {l:{} for l in FILTER_LABELS}

In [64]:
for l in FILTER_LABELS:
    print(l)
    print(len([ent for (ent, label) in chosen_ent_list if label == l]))

PERSON
2149
FAC
66
GPE
3357
NORP
602
LOC
158
EVENT
56
LANGUAGE
8
LAW
21
ORG
2226


In [ ]:
# groupby entities by their names
for data_idx, (ent, label) in enumerate(chosen_ent_list):
    chosen_ent_groupby_label[label] = {"ent": ent,
                                       "doc_idcount

In [19]:
nouse_count

15

* 완전히 같은 entity 통일하기

In [32]:
exclude_count = 0
for data_idx, data in enumerate(tqdm(xsum_test_data.dataset[:100])):
#     print(f"============ data idx: {data_idx} ============")
    document = data["document"]
    true_summary = data["true_summary"]
    
    # ner
    doc_doc = NER(document.replace("\n", " "))  # removing newline changes the ner result
    doc_sum = NER(true_summary)
    
    doc_doc_list.append(doc_doc)
    doc_sum_list.append(doc_sum)
    
    # extract entities from document
    doc_ents_filtered = [(ent.text, ent.label_) for ent in doc_doc.ents if ent.label_ in LABELS]
    doc_ents_filtered_sorted = Counter(ents_filtered).most_common()
    
    # find overlapping entities from document
    ent_overlap_flag = False
    
    for (ent, label), count in doc_ents_filtered_sorted:
        if ent in true_summary:  # there is an overlap
#             print(f"replace ent: {ent}, label: {label}, count: {count}")
            ent_overlap_flag = True
            
    
    # find overlapping entities from summary
    if ent_overlap_flag == False:
#         print("**no overlap from document**")
        
        # extract entities from summary
        sum_ents_filtered = [(ent.text, ent.label_) for ent in doc_sum.ents if ent.label_ in LABELS]
        sum_ents_filtered_sorted = Counter(sum_ents_filtered).most_common()
#         print("summary entities", sum_ents_filtered_sorted)
        
        
        if len(sum_ents_filtered_sorted) == 0: # no entity in summary -> pass
            pass
        else: # entity found in summary
            for (ent, label), count in sum_ents_filtered_sorted:
                if ent in document:  # there is an overlap from the summary
#                     print("found summary entity in the document!")
#                     print(f"replace ent: {ent}, label: {label}, count: {count}")
                    ent_overlap_flag = True
    
    if ent_overlap_flag == False:
        exclude_count += 1 # need to exclude
    

  0%|          | 0/100 [00:00<?, ?it/s]

In [35]:
doc_doc_list[0].ents

(1,099,
 2015-16,
 up to a year,
 The Welsh Government,
 the Housing Act,
 Wales,
 2015,
 Andrew Stevens,
 six months to a year,
 six months,
 Stevens,
 one,
 a hundred pounds,
 830,
 one,
 the year to March 2016,
 6,900,
 Wales,
 Marc,
 50,
 the past 20 years,
 Marc,
 Tom Clarke,
 Emmaus South Wales,
 Emmaus,
 one,
 Welsh Government,
 one,
 20,000,
 the next five years)

In [33]:
exclude_count

35

In [ ]:
xsum_test_data.dataset[idx]

In [ ]:
sum_doc = NER(["true_summary"])

# example 1

In [ ]:
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")

In [3]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, \
    headquartered in Bengaluru. It operates under Department of Space which is directly \
    overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."

In [4]:
raw_text2 = "The Mars Orbiter Mission (MOM), informally known as Mangalyaan, \
    was launched into Earth orbit on 5 November 2013 by the Indian Space Research Organisation (ISRO) \
    and has entered Mars orbit on 24 September 2014. India thus became the first country to enter \
    Mars orbit on its first attempt. It was completed at a record low cost of $74 million."

In [5]:
text = NER(raw_text2)

In [6]:
for word in text.ents:
    print(word.text,word.label_)

The Mars Orbiter Mission (MOM PRODUCT
Mangalyaan PERSON
Earth LOC
5 November 2013 DATE
the Indian Space Research Organisation ORG
Mars LOC
24 September 2014 DATE
India GPE
first ORDINAL
Mars LOC
$74 million MONEY


In [115]:
# list of ner labels
for ner_label in NER.get_pipe('ner').labels:
    print(f"* {ner_label}: {spacy.explain(ner_label)}")

* CARDINAL: Numerals that do not fall under another type
* DATE: Absolute or relative dates or periods
* EVENT: Named hurricanes, battles, wars, sports events, etc.
* FAC: Buildings, airports, highways, bridges, etc.
* GPE: Countries, cities, states
* LANGUAGE: Any named language
* LAW: Named documents made into laws.
* LOC: Non-GPE locations, mountain ranges, bodies of water
* MONEY: Monetary values, including unit
* NORP: Nationalities or religious or political groups
* ORDINAL: "first", "second", etc.
* ORG: Companies, agencies, institutions, etc.
* PERCENT: Percentage, including "%"
* PERSON: People, including fictional
* PRODUCT: Objects, vehicles, foods, etc. (not services)
* QUANTITY: Measurements, as of weight or distance
* TIME: Times smaller than a day
* WORK_OF_ART: Titles of books, songs, etc.


In [8]:
displacy.render(text,style="ent",jupyter=True)

# another example

In [49]:
# function to display basic entity info: 
def show_ents(doc):
    print(f"original doc: {doc.text}")
    if doc.ents: 
        for ent in doc.ents: 
            print(f"entity: {ent.text : >13} | start_char: {ent.start_char: 3} | end_char: {ent.end_char: 3} | label: {ent.label_} - {spacy.explain(ent.label_)}")
            # print(ent.label)  # entity type's hash value
            # print(ent.start)  # token span's start index position (word index)
            # print(ent.end)  # token span's stop index position (word index)
    else: print('No named entities found.')

In [50]:
doc1 = NER("Apple is looking at buying U.K. startup for $1 billion") 
show_ents(doc1)

original doc: Apple is looking at buying U.K. startup for $1 billion
entity:         Apple | start_char:   0 | end_char:   5 | label: ORG - Companies, agencies, institutions, etc.
entity:          U.K. | start_char:  27 | end_char:  31 | label: GPE - Countries, cities, states
entity:    $1 billion | start_char:  44 | end_char:  54 | label: MONEY - Monetary values, including unit


## document level

In [51]:
doc = NER("San Francisco considers banning sidewalk delivery robots") 
# document level 
for e in doc.ents: 
    print(e.text, e.start_char, e.end_char, e.label_) 

# OR 
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents] 
print(ents)

San Francisco 0 13 GPE
[('San Francisco', 0, 13, 'GPE')]


## token level

In [52]:
# token level 
# doc[0], doc[1] ...will have tokens stored. 

ent_san = [doc[0].text, doc[0].ent_iob_, doc[0].ent_type_] 
ent_francisco = [doc[1].text, doc[1].ent_iob_, doc[1].ent_type_]
ent_considers = [doc[2].text, doc[2].ent_iob_, doc[2].ent_type_] 
print(ent_san) 
print(ent_francisco)
print(ent_considers)

# token.ent_iob indicates whether an entity starts continues or ends on the tag
# I - Token is inside an entity. 
# O - Token is outside an entity. 
# B - Token is the beginning of an entity.

['San', 'B', 'GPE']
['Francisco', 'I', 'GPE']
['considers', 'O', '']


## User-Defined Named Entity and Adding it to a Span

### Example 1

In [53]:
doc = NER(u'Tesla to build a U.K. factory for $6 million.')
show_ents(doc)

original doc: Tesla to build a U.K. factory for $6 million.
entity:          U.K. | start_char:  17 | end_char:  21 | label: GPE - Countries, cities, states
entity:    $6 million | start_char:  34 | end_char:  44 | label: MONEY - Monetary values, including unit


In [54]:
from spacy.tokens import Span

In [55]:
# get the hash value of the ORG entity label
ORG = doc.vocab.strings[u"ORG"]
print(ORG)

# create a span for the new entity
new_ent = Span(doc=doc, start=0, end=1, label=ORG)
print(new_ent)

# add the entity to the existing doc object
doc.ents = list(doc.ents) + [new_ent]
print(doc.ents)

383
Tesla
(Tesla, U.K., $6 million)


In [56]:
show_ents(doc)

original doc: Tesla to build a U.K. factory for $6 million.
entity:         Tesla | start_char:   0 | end_char:   5 | label: ORG - Companies, agencies, institutions, etc.
entity:          U.K. | start_char:  17 | end_char:  21 | label: GPE - Countries, cities, states
entity:    $6 million | start_char:  34 | end_char:  44 | label: MONEY - Monetary values, including unit


## Adding Named Entities to All Matching Spans

In [57]:
doc = NER(u'Our company plans to introduce a new vacuum cleaner. If successful, the vacuum cleaner will be our first product.') 
show_ents(doc) 

original doc: Our company plans to introduce a new vacuum cleaner. If successful, the vacuum cleaner will be our first product.
entity:         first | start_char:  99 | end_char:  104 | label: ORDINAL - "first", "second", etc.


In [58]:
# Import PhraseMatcher and create a matcher object: 
from spacy.matcher import PhraseMatcher 
matcher = PhraseMatcher(NER.vocab)

In [59]:
# Create the desired phrase patterns:
phrase_list = ['vacuum cleaner', 'vacuum-cleaner']
phrase_patterns = [NER(text) for text in phrase_list]
print(phrase_list)
print(phrase_patterns)

['vacuum cleaner', 'vacuum-cleaner']
[vacuum cleaner, vacuum-cleaner]


In [60]:
# Apply the patterns to our matcher object:
matcher.add('newproduct', None, *phrase_patterns)

In [61]:
# Apply the matcher to our Doc object:
matches = matcher(doc)
#See what matches occur: 
matches 

[(2689272359382549672, 7, 9), (2689272359382549672, 14, 16)]

In [62]:
# Here we create Spans from each match, and create named entities from them: 
from spacy.tokens import Span 
PROD = doc.vocab.strings[u'PRODUCT'] 
new_ents = [Span(doc, match[1], match[2],label=PROD) for match in matches]
new_ents

[vacuum cleaner, vacuum cleaner]

In [63]:
# match[1] contains the start index of the the token and match[2] the stop index (exclusive) of the token in the doc. 
doc.ents = list(doc.ents) + new_ents 
show_ents(doc)

original doc: Our company plans to introduce a new vacuum cleaner. If successful, the vacuum cleaner will be our first product.
entity: vacuum cleaner | start_char:  37 | end_char:  51 | label: PRODUCT - Objects, vehicles, foods, etc. (not services)
entity: vacuum cleaner | start_char:  72 | end_char:  86 | label: PRODUCT - Objects, vehicles, foods, etc. (not services)
entity:         first | start_char:  99 | end_char:  104 | label: ORDINAL - "first", "second", etc.


## counting entities

In [64]:
doc = NER(u"originally priced at $29.50, now it's marked down to five dollars")
show_ents(doc)

original doc: originally priced at $29.50, now it's marked down to five dollars
entity:         29.50 | start_char:  22 | end_char:  27 | label: MONEY - Monetary values, including unit
entity:  five dollars | start_char:  53 | end_char:  65 | label: MONEY - Monetary values, including unit


In [65]:
len([ent for ent in doc.ents if ent.label_ == "MONEY"])

2

## Visualizing NER

In [ ]:
from spacy import displacy

In [72]:
doc = NER(u"Tesla to build a U.K. factory for $6 million. "
          u"originally priced at $29.50, now it's marked down to five dollars")
displacy.render(doc, style="ent", jupyter=True)


In [74]:
# line by line
for sent in doc.sents:
    displacy.render(sent, style="ent", jupyter=True)

In [79]:
# viewing specific entries
options = {"ents": ["MONEY"]}
displacy.render(doc, style="ent", jupyter=True, options = options)